In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "True"

# Summarization

In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_openai import ChatOpenAI

# Path to your .txt file
file_path = "/home/david/Documents/hackathon2024/data/transcription_call_8.txt"

# loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
loader = TextLoader(file_path)
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(page_content=" No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years old and I have a dog in the house. Okay, there, there, okay. Let me get something right over to you. Did you go in your mommy and daddy's room? Oh, and there's a blood. Oh, over the place? Not all over. There's blood on the plant and blood on the floor. Oh, my goodness. You have your little doggy with you? And three cats. Well, get some cats too. Three cats and one dog. Okay. Are you the only one there? Is this like mommy and daddy? Oh, my... It's dad and mommy and daddy and they didn't even answer. Okay. Okay, what I want you to do, honey, I w

In [21]:
texts = docs[0].page_content
print(texts)

 No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years old and I have a dog in the house. Okay, there, there, okay. Let me get something right over to you. Did you go in your mommy and daddy's room? Oh, and there's a blood. Oh, over the place? Not all over. There's blood on the plant and blood on the floor. Oh, my goodness. You have your little doggy with you? And three cats. Well, get some cats too. Three cats and one dog. Okay. Are you the only one there? Is this like mommy and daddy? Oh, my... It's dad and mommy and daddy and they didn't even answer. Okay. Okay, what I want you to do, honey, I want you to stand up for 

In [3]:
from langchain.chains.summarize import load_summarize_chain
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
local_llm = "gpt-4-0125-preview"
llm = ChatOpenAI(
    model = local_llm,
    base_url = "http://10.128.138.175:11434/v1/",
    temperature=0,
    api_key="ollama",
    streaming=True,
)
chain = load_summarize_chain(llm, chain_type="stuff")

result = chain.invoke(docs)

print(result["output_text"])

A 5-year-old child calls 911 after finding their parents dead in their home. The child is scared and confused, but tries to provide information to the dispatcher about what happened. They mention seeing a bullet on the floor and a slide coming out of their father's mouth. The child also talks about their dog, Liza, and mentions that they were sleeping when they heard a noise. The dispatcher tries to reassure the child and asks them to stand up for her at the front door, where an officer will be waiting.


In [4]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
output = stuff_chain.invoke(docs)["output_text"]
print(output)

/home/david/Documents/cira-project/code-cira/tools-langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


A 5-year-old child calls 911 after finding their parents dead in their home. The child is shaken and confused, but tries to provide information about the situation. They report seeing a bullet on the floor and a slide coming out of their father's mouth. The operator tries to comfort the child and gather more information, including the location of the house and the presence of pets (a dog named Liza and three cats). The child is eventually instructed to open the front door for an officer who will arrive shortly.


# Tagging

In [5]:
# Path to your model weights
local_model = "/home/david/Documents/cira-project/llmodel/ggufModel/model/llama-2-7b-chat-dpo.gguf"

In [6]:
import multiprocessing

from langchain_community.chat_models import ChatLlamaCpp

llm_cpp = ChatLlamaCpp(
    temperature=0.5,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=8,
    n_batch=300,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    max_tokens=512,
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.5,
    top_p=0.5,
    verbose=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/david/Documents/cira-project/llmodel/ggufModel/model/llama-2-7b-chat-dpo.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   

llm_load_tensors:        CPU buffer size =  6828.64 MiB
...................................................................................................
llama_new_context_with_model: n_ctx      = 10016
llama_new_context_with_model: n_batch    = 300
llama_new_context_with_model: n_ubatch   = 300
llama_new_context_with_model: flash_attn = 0
llama_new_context_with_model: freq_base  = 10000.0
llama_new_context_with_model: freq_scale = 1
llama_kv_cache_init:        CPU KV buffer size =  5008.00 MiB
llama_new_context_with_model: KV self size  = 5008.00 MiB, K (f16): 2504.00 MiB, V (f16): 2504.00 MiB
llama_new_context_with_model:        CPU  output buffer size =     0.12 MiB
llama_new_context_with_model:        CPU compute buffer size =   397.78 MiB
llama_new_context_with_model: graph nodes  = 1030
llama_new_context_with_model: graph splits = 1
AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 |

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'IncidentClassification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")

class IncidentClassification(BaseModel):
    violence: str = Field(description="Whether the incident involved violence")
    fire: str = Field(description="Whether the incident involved fire")
    medical: str = Field(description="Whether the incident involved medical attention")      
    suspicious_activity: str = Field(description="Whether the incident involved suspicious activity")                                                                                         


# LLM
local_llm = "gpt-4-0125-preview"
llm_original = ChatOpenAI(
    model = local_llm,
    base_url = "http://10.128.138.175:11434/v1/",
    temperature=0,
    api_key="ollama",
    streaming=True,
)
# llm = llm_original.with_structured_output(
#     IncidentClassification
# )

llm = llm_cpp.with_structured_output(
    IncidentClassification
)

tagging_chain = tagging_prompt | llm

In [8]:
output

"A 5-year-old child calls 911 after finding their parents dead in their home. The child is shaken and confused, but tries to provide information about the situation. They report seeing a bullet on the floor and a slide coming out of their father's mouth. The operator tries to comfort the child and gather more information, including the location of the house and the presence of pets (a dog named Liza and three cats). The child is eventually instructed to open the front door for an officer who will arrive shortly."

In [9]:
# inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = tagging_chain.invoke({"input": output})
res

from_string grammar:
char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
fire-kv ::= ["] [f] [i] [r] [e] ["] space [:] space string 
space ::= space_9 
string ::= ["] string_10 ["] space 
medical-kv ::= ["] [m] [e] [d] [i] [c] [a] [l] ["] space [:] space string 
root ::= [{] space violence-kv [,] space fire-kv [,] space medical-kv [,] space suspicious-activity-kv [}] space 
violence-kv ::= ["] [v] [i] [o] [l] [e] [n] [c] [e] ["] space [:] space string 
suspicious-activity-kv ::= ["] [s] [u] [s] [p] [i] [c] [i] [o] [u] [s] [_] [a] [c] [t] [i] [v] [i] [t] [y] ["] space [:] space string 
space_9 ::= [ ] | 
string_10 ::= char string_10 | 




llama_print_timings:        load time =    7958.00 ms
llama_print_timings:      sample time =      86.66 ms /    33 runs   (    2.63 ms per token,   380.79 tokens per second)
llama_print_timings: prompt eval time =    7957.42 ms /   155 tokens (   51.34 ms per token,    19.48 tokens per second)
llama_print_timings:        eval time =    8467.33 ms /    32 runs   (  264.60 ms per token,     3.78 tokens per second)
llama_print_timings:       total time =   16547.35 ms /   187 tokens


IncidentClassification(violence='bullets', fire='', medical='', suspicious_activity='')

In [10]:
info_tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'IncidentInformation' function.

Passage:
{input}
"""
)

class IncidentInformation(BaseModel):
    who: str = Field(description="Who was involved in the incident")
    what: str = Field(description="What happened during the incident")
    when: str = Field(description="When did the incident occur")
    where: str = Field(description="Where did the incident occur")
    why: str = Field(description="Why did the incident occur")


info_llm = llm_cpp.with_structured_output(
    IncidentInformation
)

info_tagging_chain = info_tagging_prompt | info_llm

In [11]:
docs 

[Document(page_content=" No one works for emergency. Um, hello. Hello. Is everything okay? Um, my mommy and daddy. Uh-huh. I think there's a bullet on the floor. Is there the what? And there's a... There's a slide coming out of my dad's mouth and he fell off the bed. He did? Where's mommy at? I... I don't know. I think they're dead. What do you mean, sweetheart? I don't know. Okay, dude. Your daddy's on the floor. How old are you? I'm five years old and I have a dog in the house. Okay, there, there, okay. Let me get something right over to you. Did you go in your mommy and daddy's room? Oh, and there's a blood. Oh, over the place? Not all over. There's blood on the plant and blood on the floor. Oh, my goodness. You have your little doggy with you? And three cats. Well, get some cats too. Three cats and one dog. Okay. Are you the only one there? Is this like mommy and daddy? Oh, my... It's dad and mommy and daddy and they didn't even answer. Okay. Okay, what I want you to do, honey, I w

In [12]:
# inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
info_res = info_tagging_chain.invoke({"input": docs})
info_res

from_string grammar:
char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
root ::= [{] space who-kv [,] space what-kv [,] space when-kv [,] space where-kv [,] space why-kv [}] space 
space ::= space_9 
who-kv ::= ["] [w] [h] [o] ["] space [:] space string 
what-kv ::= ["] [w] [h] [a] [t] ["] space [:] space string 
when-kv ::= ["] [w] [h] [e] [n] ["] space [:] space string 
where-kv ::= ["] [w] [h] [e] [r] [e] ["] space [:] space string 
why-kv ::= ["] [w] [h] [y] ["] space [:] space string 
space_9 ::= [ ] | 
string ::= ["] string_11 ["] space 
string_11 ::= char string_11 | 

Llama.generate: prefix-match hit



llama_print_timings:        load time =    7958.00 ms
llama_print_timings:      sample time =     130.59 ms /    46 runs   (    2.84 ms per token,   352.24 tokens per second)
llama_print_timings: prompt eval time =   71000.77 ms /  1306 tokens (   54.37 ms per token,    18.39 tokens per second)
llama_print_timings:        eval time =   12921.49 ms /    45 runs   (  287.14 ms per token,     3.48 tokens per second)
llama_print_timings:       total time =   84107.13 ms /  1351 tokens


IncidentInformation(who='mommy and daddy', what='gun shot', when='tonight', where='the house', why='I don’t know')

# Long-Context Reorder

reduce the important information degradation

In [22]:
# %pip install --upgrade --quiet  sentence-transformers langchain-chroma langchain langchain-openai > /dev/null

In [23]:
# from langchain.chains import LLMChain, StuffDocumentsChain
# from langchain_chroma import Chroma
# from langchain_community.document_transformers import (
#     LongContextReorder,
# )
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_core.prompts import PromptTemplate
# from langchain_openai import OpenAI

# # Get embeddings.
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # texts = [
# #     "Basquetball is a great sport.",
# #     "Fly me to the moon is one of my favourite songs.",
# #     "The Celtics are my favourite team.",
# #     "This is a document about the Boston Celtics",
# #     "I simply love going to the movies",
# #     "The Boston Celtics won the game by 20 points",
# #     "This is just a random text.",
# #     "Elden Ring is one of the best games in the last 15 years.",
# #     "L. Kornet is one of the best Celtics players.",
# #     "Larry Bird was an iconic NBA player.",
# # ]

# # Create a retriever
# retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
#     search_kwargs={"k": 10}
# )


/home/david/Documents/cira-project/code-cira/tools-langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/david/Documents/cira-project/code-cira/tools-langchain/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/david/Documents/cira-project/code-cira/tools-langchain/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarn

In [24]:
# retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x74b3505217c0>, search_kwargs={'k': 10})

In [27]:
# query = "What can you tell me about the Celtics?"

# # Get relevant documents ordered by relevance score
# docs = retriever.invoke("summarize")
# docs

[Document(page_content='s'),
 Document(page_content='s'),
 Document(page_content='s'),
 Document(page_content='s'),
 Document(page_content='s'),
 Document(page_content='s'),
 Document(page_content='S'),
 Document(page_content='s'),
 Document(page_content='s'),
 Document(page_content='s')]